In [148]:
import pandas as pd
from statsmodels.stats.power import FTestAnovaPower
import scipy.stats as stats
from collections import defaultdict
import numpy as np
import math

Power Analysis

In [124]:
def cohens_f(f_stat, df_groups, df_obs):
    
    eta_squared = (f_stat * df_groups) / (f_stat * df_groups + df_obs)
    cohens_f = np.sqrt(eta_squared / (1 - eta_squared))
    return cohens_f


In [17]:
game_output_data = pd.read_csv(r"ISYE 6644/project/C\:\\Users\\Will\\Documents\\school_data\\gol.csv")

In [46]:
variable_summary_output_df = game_output_data.groupby(["matrix_size","initial_percent_alive","run"],as_index=False)["gen"].max()


In [ ]:
anova_results = []
for size in variable_summary_output_df["matrix_size"].unique():
    size_df = variable_summary_output_df[variable_summary_output_df["matrix_size"]==size]
    percent_alive_grouped_df = size_df.groupby(["initial_percent_alive","run"],as_index=False)["gen"].max()
    grouped_lists = percent_alive_grouped_df.groupby("initial_percent_alive")["gen"].apply(list)

    f_stat, p_value = stats.f_oneway(*grouped_lists)
    
    degree_of_freedom_groups = len(grouped_lists)-1
    degree_of_freedom_observations = size_df.shape[0] - len(grouped_lists)
    con_f = cohens_f(f_stat,degree_of_freedom_groups,degree_of_freedom_observations)
    anova_results.append({
        "matrix_size": size,
        "f_stat": f_stat,
        "p_value": p_value,
        "cohen_f":con_f
    })
    

19

In [130]:
anova_analysis_df = pd.DataFrame(anova_results)
anova_analysis_df

,matrix_size,f_stat,p_value,cohen_f
0,100,4.800275,9.036791e-09,0.674361
1,400,15.969233,4.181975e-29,1.229990
2,900,41.474539,6.083548e-56,1.982213
3,1600,39.418029,2.581696e-54,1.932444
4,2500,37.450042,1.075697e-52,1.883587
5,3600,53.304607,3.086617e-64,2.247201
6,4900,76.949741,4.698422e-77,2.699995
7,6400,62.820664,7.007767e-70,2.439556
8,8100,54.586706,4.816261e-65,2.274065
9,10000,37.941779,4.178323e-53,1.895913


In [156]:
sample_size_dict = {}
for _,r in anova_analysis_df.iterrows():
    effect = float(r["cohen_f"])
    if effect>=1:
        effect=.98
    power_analysis = FTestAnovaPower()
    z= power_analysis.solve_power(effect_size=effect, alpha=.05, power=.9, k_groups=19)
    sample_size_dict[int(r["matrix_size"]/10)]=  (math.ceil(z))

In [157]:
sample_size_dict

{10: 72,
 40: 44,
 90: 44,
 160: 44,
 250: 44,
 360: 44,
 490: 44,
 640: 44,
 810: 44,
 1000: 44}

In [90]:
grouped = variable_summary_output_df.groupby(["matrix_size","initial_percent_alive"])["gen"].apply(list)

In [92]:
f_stat, p_value = stats.f_oneway(*grouped)

In [93]:
f_stat

np.float64(41.0008386812046)

Based on the average number of generations for each variable combination, when running a anova one way variance test indicates there is there is a significant different between those group mean. Which tells us that there is a different between the groups when it comes to generation size.
F stat: 41

The null hypothese for this these is; There is not a significant different between the groups. With a p-value of less than .05, we can we can reject the null hypthoese is favor of the alternative, which is there is a significant different between the groups

In [ ]:
power_analysis = FTestAnovaPower()



In [145]:

effect_size = 0.674361  # Example: small effect size (Cohen's f)
alpha = 0.05  # Significance level
power = 0.80  # Desired power

# Create the power analysis object
power_analysis = FTestAnovaPower()

# Calculate the required sample size
sample_size = power_analysis.solve_power(effect_size=effect_size, alpha=alpha, power=power, k_groups=19)  # k_groups is the number of groups

# Print the sample size
print(f"Required sample size per group: {round(sample_size)}")


Required sample size per group: 60


In [19]:
variable_summary_output_df.groupby(["matrix_size","initial_percent_alive"],as_index=False).agg({ "gen": ["mean", "std"]})

matrix_size initial_percent_alive        gen           
                                            mean        std
0           100                  0.05   1.181818   0.404520
1           100                  0.10  21.363636  38.761509
2           100                  0.15  17.272727  22.347666
3           100                  0.20  42.000000  39.547440
4           100                  0.25  30.545455  26.934601
..          ...                   ...        ...        ...
185       10000                  0.75  99.000000   0.000000
186       10000                  0.80  82.272727  37.226579
187       10000                  0.85  41.181818  45.860262
188       10000                  0.90  13.272727  28.471357
189       10000                  0.95   2.272727   0.467099

[190 rows x 4 columns]